## 止盈止损-固定价位止损止盈

1. 固定点数(Point)
2. 固定百分比(Percentage)
3. 固定平保(Protect)

### 固定点数(Point)
1. 设置止损止盈的点数
2. 成交时需要记录成交价格
3. 检查当前是持有多头还是空头
4. 设置当前价格触发成交价‘加减’固定点数的止损止盈时执行平仓
5. 平仓的数量为当前所有多头或空头的持仓数量

In [ ]:
# 设置参数

stopLossPoint = 0.1 #1毛
takeProfitPoint = 0.3 #3毛

# 计算止损止盈价位
longStop = self.transactionPrice*(1-self.stopLossPct)
longProfit = self.transactionPrice*(1+self.takeProfitPct)
shortStop = self.transactionPrice*(1-self.stopLossPct)
shortProfit = self.transactionPrice*(1+self.takeProfitPct)

# 持有多头
if (self.posDict[symbol+'_LONG'] > 0):
    if (bar.close<=longStop) or (bar.close>=longProfit):
        self.sell(symbol,bar.close*0.98, self.posDict[symbol+'_LONG'])

# 持有空头
elif (self.posDict[symbol+'_SHORT'] > 0):
    if (bar.close>shortStop) or (bar.close<takeProfitPoint):
        self.cover(symbol,bar.close*1.02, self.posDict[symbol+'_SHORT'])

### 固定百分比(Percentage)
1. 设置止损止盈的百分比
2. 成交时需要记录成交价格
3. 检查当前是持有多头还是空头
4. 设置当前价格触发成交价‘乘以’固定百分比的止损止盈时执行平仓
5. 平仓的数量为当前所有多头或空头的持仓数量

In [ ]:
# 设置参数
stopLossPct = 0.01 #百分之一
takeProfitPct = 0.03 #百分之三

# 计算止损止盈价位
longStop = self.transactionPrice*(1-self.stopLossPct)
longProfit = self.transactionPrice*(1+self.takeProfitPct)
shortStop = self.transactionPrice*(1-self.stopLossPct)
shortProfit = self.transactionPrice*(1+self.takeProfitPct)

# 持有多头
if (self.posDict[symbol+'_LONG'] > 0):
    if (bar.close<= longStop) or (bar.close>=longProfit):
        self.sell(symbol,bar.close*0.98, self.posDict[symbol+'_LONG'])

# 持有空头
elif (self.posDict[symbol+'_SHORT'] > 0):
    if (bar.close>shortStop) or (bar.close<shortProfit):
        self.cover(symbol,bar.close*1.02, self.posDict[symbol+'_SHORT'])

### 固定平保(Protect)
有三种出场可能，止损，平保，止盈。（其中止损和平保互斥）

1. 设置止损止盈的百分比，并且设置平保的百分比
2. 成交时需要记录成交价格
3. 检查当前是持有多头还是空头
4. 设置平保变量，满足条件时设置为1，若没满足平保条件则按正常止损比例止损
5. 设置止盈出场的比例与满足条件，满足下离场
5. 平仓的数量为当前所有多头或空头的持仓数量

In [ ]:
# 设置参数
stopLossPct = 0.01; takeProfitPct=0.3; protectPct=0.1

# 计算止损止盈价位
buyTakeProfitPrice = self.transactionPrice[symbol] * (1 + self.takeProfitPct)
sellTakeProfitPrice = self.transactionPrice[symbol] * (1 - self.takeProfitPct)
buyStopLossPrice = self.transactionPrice[symbol] * (1 - self.stopLossPct)
sellStopLossPrice = self.transactionPrice[symbol] * (1 + self.stopLossPct)
buyProtectStopPrice = self.transactionPrice[symbol]*(1+self.protectPct)
sellProtectStopPrice = self.transactionPrice[symbol]*(1-self.protectPct)

if (self.posDict[symbol + '_LONG'] > 0):
    #启动平保
    if (bar.close )>= buyProtectStopPrice:
        self.stopProtect[symbol] = 1 
    #满足平保条件
    if self.stopProtect[symbol] == 1:
        if bar.close <= (1.002 * self.transactionPrice[symbol]):
            self.sell(symbol, bar.close * 0.98, self.posDict[symbol + '_LONG'])
            self.stopProtect[symbol] = 0
    #满足止损条件
    elif (self.stopProtect[symbol] == 0) and (bar.close <= buyStopLossPrice):
            self.sell(symbol, bar.close * 0.98, self.posDict[symbol + '_LONG'])
    #满足止盈条件
    if (bar.close >= buyTakeProfitPrice):
        self.sell(symbol, bar.close * 0.98, self.posDict[symbol + '_LONG'])
        self.stopProtect[symbol] = 0

elif (self.posDict[symbol + '_SHORT'] > 0):
    #启动平保
    if bar.close <= sellProtectStopPrice:
        self.stopProtect[symbol] = -1
    #满足平保条件
    if (self.stopProtect[symbol] == -1) and (bar.close >= (0.998 * self.transactionPrice[symbol])):
            self.cover(symbol, bar.close * 1.02, self.posDict[symbol + '_SHORT'])
            self.stopProtect[symbol] = 0
    #满足止损条件
    elif (self.stopProtect[symbol] == 0) and (bar.close >= sellStopLossPrice):
            self.cover(symbol, bar.close * 1.02, self.posDict[symbol + '_SHORT'])
    #满足止盈条件
    if (bar.close <= sellTakeProfitPrice):
        self.cover(symbol, bar.close * 1.02, self.posDict[symbol + '_SHORT'])
        self.stopProtect[symbol] = 0

#空仓时将平保条件还原为0
elif (self.posDict[symbol + '_LONG'] == 0) and (self.posDict[symbol + '_SHORT'] == 0):
    self.stopProtect[symbol] = 0